In [1]:
import numpy as np
import sympy
from sympy.abc import alpha, x, y, v, w, R, theta
from sympy import symbols, Matrix

In [8]:
px = symbols('p_{x}')
py = symbols('p_{y}')
pz = symbols('p_{z}')

vx = symbols('v_{x}')
vy = symbols('v_{y}')
vz = symbols('v_{z}')

q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')

ax = symbols('a_{x}')
ay = symbols('a_{y}')
az = symbols('a_{z}')
wx = symbols('w_{x}')
wy = symbols('w_{y}')
wz = symbols('w_{z}')

dt = symbols('dt')
norm_w = symbols('\|w\|')

g = Matrix([
    [0],[0],[9.81]
])

In [3]:
R = Matrix([[q1**2 + q2**2 - q3**2 - q4**2, 2*(q2*q3 - q1*q4), 2*(q1*q3 + q2*q4)],
            [2*(q2*q3 + q1*q4), q1**2 - q2**2 + q3**2 - q4**2, 2*(q3*q4 - q1*q2)],
            [2*(q2*q4 - q1*q3), 2*(q1*q2 + q3*q4), q1**2 - q2**2 - q3**2 + q4**2]
           ])
R

Matrix([
[q1**2 + q2**2 - q3**2 - q4**2,            -2*q1*q4 + 2*q2*q3,             2*q1*q3 + 2*q2*q4],
[            2*q1*q4 + 2*q2*q3, q1**2 - q2**2 + q3**2 - q4**2,            -2*q1*q2 + 2*q3*q4],
[           -2*q1*q3 + 2*q2*q4,             2*q1*q2 + 2*q3*q4, q1**2 - q2**2 - q3**2 + q4**2]])

In [5]:
values={
    q1: 1, 
    q2: 0, 
    q3: 0, 
    q4: 0
}
np.array(R.subs(values)).astype(float)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [6]:
Omega = Matrix([
    [0, wz, -wy, wx],
    [-wz, 0, wx, wy],
    [wy, -wx, 0, wz],
    [-wx, -wy, -wz, 0]
])

In [7]:
values={
    wx: 0.5, 
    wy: 0.5, 
    wz: 0.5, 
}
omega_ = np.array(Omega.subs(values)).astype(float)
omega_

array([[ 0. ,  0.5, -0.5,  0.5],
       [-0.5,  0. ,  0.5,  0.5],
       [ 0.5, -0.5,  0. ,  0.5],
       [-0.5, -0.5, -0.5,  0. ]])

In [9]:
A = sympy.cos(norm_w*dt/2) * sympy.eye(4)
B = (1/norm_w)*sympy.sin(norm_w*dt/2)
qk = Matrix([
    [q1],
    [q2],
    [q3],
    [q4]
])
(A + B * Omega) * qk

Matrix([
[q1*cos(\|w\|*dt/2) + q2*w_{z}*sin(\|w\|*dt/2)/\|w\| - q3*w_{y}*sin(\|w\|*dt/2)/\|w\| + q4*w_{x}*sin(\|w\|*dt/2)/\|w\|],
[q2*cos(\|w\|*dt/2) - q1*w_{z}*sin(\|w\|*dt/2)/\|w\| + q3*w_{x}*sin(\|w\|*dt/2)/\|w\| + q4*w_{y}*sin(\|w\|*dt/2)/\|w\|],
[q3*cos(\|w\|*dt/2) + q1*w_{y}*sin(\|w\|*dt/2)/\|w\| - q2*w_{x}*sin(\|w\|*dt/2)/\|w\| + q4*w_{z}*sin(\|w\|*dt/2)/\|w\|],
[q4*cos(\|w\|*dt/2) - q1*w_{x}*sin(\|w\|*dt/2)/\|w\| - q2*w_{y}*sin(\|w\|*dt/2)/\|w\| - q3*w_{z}*sin(\|w\|*dt/2)/\|w\|]])

In [10]:
wx_, wy_, wz_ =  0.5, 0.5, 0.5
norm_w_ = np.sqrt(wx_**2 + wy_**2 + wz_**2)
values = { 
    norm_w: norm_w_, 
    dt: 0.1
}
qk_values = {
    q1: 1, 
    q2: 0,
    q3: 0,
    q4: 0
}

qk_ = np.array((A.subs(values) + B.subs(values) * omega_) * qk.subs(qk_values)).astype(float)
qk_

array([[ 0.99906265],
       [-0.02499219],
       [ 0.02499219],
       [-0.02499219]])

In [11]:
fxu = Matrix([
    [Matrix([[px],[py], [pz]]) +  Matrix([[vx],[vy], [vz]]) * dt + (R * Matrix([[ax],[ay], [az]]) - g)*dt**2 / 2],
    [Matrix([[vx],[vy], [vz]]) + (R * Matrix([[ax],[ay], [az]]) - g) * dt],
    [(A + B * Omega) * qk],
])
fxu

Matrix([
[       dt**2*(a_{x}*(q1**2 + q2**2 - q3**2 - q4**2) + a_{y}*(-2*q1*q4 + 2*q2*q3) + a_{z}*(2*q1*q3 + 2*q2*q4))/2 + dt*v_{x} + p_{x}],
[       dt**2*(a_{x}*(2*q1*q4 + 2*q2*q3) + a_{y}*(q1**2 - q2**2 + q3**2 - q4**2) + a_{z}*(-2*q1*q2 + 2*q3*q4))/2 + dt*v_{y} + p_{y}],
[dt**2*(a_{x}*(-2*q1*q3 + 2*q2*q4) + a_{y}*(2*q1*q2 + 2*q3*q4) + a_{z}*(q1**2 - q2**2 - q3**2 + q4**2) - 9.81)/2 + dt*v_{z} + p_{z}],
[                       dt*(a_{x}*(q1**2 + q2**2 - q3**2 - q4**2) + a_{y}*(-2*q1*q4 + 2*q2*q3) + a_{z}*(2*q1*q3 + 2*q2*q4)) + v_{x}],
[                       dt*(a_{x}*(2*q1*q4 + 2*q2*q3) + a_{y}*(q1**2 - q2**2 + q3**2 - q4**2) + a_{z}*(-2*q1*q2 + 2*q3*q4)) + v_{y}],
[                dt*(a_{x}*(-2*q1*q3 + 2*q2*q4) + a_{y}*(2*q1*q2 + 2*q3*q4) + a_{z}*(q1**2 - q2**2 - q3**2 + q4**2) - 9.81) + v_{z}],
[             q1*cos(\|w\|*dt/2) + q2*w_{z}*sin(\|w\|*dt/2)/\|w\| - q3*w_{y}*sin(\|w\|*dt/2)/\|w\| + q4*w_{x}*sin(\|w\|*dt/2)/\|w\|],
[             q2*cos(\|w\|*dt/2) - q1*w_{z}*sin(\|w\|

In [16]:
ax_, ay_, az_ = 1, 0, 0
px_, py_, pz_ = 0, 0, 0
wx_, wy_, wz_ =  0.5, 0.5, 0.5
values = {
    dt: 0.1,
    px: px_,
    py: py_,
    pz: pz_,
    vx: 0,
    vy: 0,
    vz: 0,
    q1: 1, 
    q2: 0,
    q3: 0,
    q4: 0,
    ax: ax_,
    ay: ay_,
    az: az_,
    wx: wx_,
    wy: wy_, 
    wz: wz_,
    norm_w: norm_w_
}
fxu.evalf(subs=values)

Matrix([
[              0.005],
[                  0],
[           -0.04905],
[                0.1],
[                  0],
[             -0.981],
[   0.99906264647522],
[-0.0249921882323892],
[ 0.0249921882323892],
[-0.0249921882323892]])

In [14]:
state_x = Matrix([px, py, pz, vx, vy, vz, q1, q2, q3, q4])
control_input = Matrix([ax, ay, az, wx, wy, wz])

In [15]:
#compute Jcobian
F = fxu.jacobian(state_x)
F

Matrix([
[1, 0, 0, dt,  0,  0,  dt**2*(2*a_{x}*q1 - 2*a_{y}*q4 + 2*a_{z}*q3)/2, dt**2*(2*a_{x}*q2 + 2*a_{y}*q3 + 2*a_{z}*q4)/2, dt**2*(-2*a_{x}*q3 + 2*a_{y}*q2 + 2*a_{z}*q1)/2, dt**2*(-2*a_{x}*q4 - 2*a_{y}*q1 + 2*a_{z}*q2)/2],
[0, 1, 0,  0, dt,  0,  dt**2*(2*a_{x}*q4 + 2*a_{y}*q1 - 2*a_{z}*q2)/2, dt**2*(2*a_{x}*q3 - 2*a_{y}*q2 - 2*a_{z}*q1)/2,  dt**2*(2*a_{x}*q2 + 2*a_{y}*q3 + 2*a_{z}*q4)/2,  dt**2*(2*a_{x}*q1 - 2*a_{y}*q4 + 2*a_{z}*q3)/2],
[0, 0, 1,  0,  0, dt, dt**2*(-2*a_{x}*q3 + 2*a_{y}*q2 + 2*a_{z}*q1)/2, dt**2*(2*a_{x}*q4 + 2*a_{y}*q1 - 2*a_{z}*q2)/2, dt**2*(-2*a_{x}*q1 + 2*a_{y}*q4 - 2*a_{z}*q3)/2,  dt**2*(2*a_{x}*q2 + 2*a_{y}*q3 + 2*a_{z}*q4)/2],
[0, 0, 0,  1,  0,  0,       dt*(2*a_{x}*q1 - 2*a_{y}*q4 + 2*a_{z}*q3),      dt*(2*a_{x}*q2 + 2*a_{y}*q3 + 2*a_{z}*q4),      dt*(-2*a_{x}*q3 + 2*a_{y}*q2 + 2*a_{z}*q1),      dt*(-2*a_{x}*q4 - 2*a_{y}*q1 + 2*a_{z}*q2)],
[0, 0, 0,  0,  1,  0,       dt*(2*a_{x}*q4 + 2*a_{y}*q1 - 2*a_{z}*q2),      dt*(2*a_{x}*q3 - 2*a_{y}*q2 - 2*a_{z}*q1), 

In [17]:
F.evalf(subs=values)

Matrix([
[1.0,   0,   0, 0.1,   0,   0,                0.01,                   0,                   0,                  0],
[  0, 1.0,   0,   0, 0.1,   0,                   0,                   0,                   0,               0.01],
[  0,   0, 1.0,   0,   0, 0.1,                   0,                   0,               -0.01,                  0],
[  0,   0,   0, 1.0,   0,   0,                 0.2,                   0,                   0,                  0],
[  0,   0,   0,   0, 1.0,   0,                   0,                   0,                   0,                0.2],
[  0,   0,   0,   0,   0, 1.0,                   0,                   0,                -0.2,                  0],
[  0,   0,   0,   0,   0,   0,    0.99906264647522,  0.0249921882323892, -0.0249921882323892, 0.0249921882323892],
[  0,   0,   0,   0,   0,   0, -0.0249921882323892,    0.99906264647522,  0.0249921882323892, 0.0249921882323892],
[  0,   0,   0,   0,   0,   0,  0.0249921882323892, -0.0249921882323892

In [18]:
G = fxu.jacobian(control_input)
G

Matrix([
[dt**2*(q1**2 + q2**2 - q3**2 - q4**2)/2,            dt**2*(-2*q1*q4 + 2*q2*q3)/2,             dt**2*(2*q1*q3 + 2*q2*q4)/2,                         0,                         0,                         0],
[            dt**2*(2*q1*q4 + 2*q2*q3)/2, dt**2*(q1**2 - q2**2 + q3**2 - q4**2)/2,            dt**2*(-2*q1*q2 + 2*q3*q4)/2,                         0,                         0,                         0],
[           dt**2*(-2*q1*q3 + 2*q2*q4)/2,             dt**2*(2*q1*q2 + 2*q3*q4)/2, dt**2*(q1**2 - q2**2 - q3**2 + q4**2)/2,                         0,                         0,                         0],
[     dt*(q1**2 + q2**2 - q3**2 - q4**2),                 dt*(-2*q1*q4 + 2*q2*q3),                  dt*(2*q1*q3 + 2*q2*q4),                         0,                         0,                         0],
[                 dt*(2*q1*q4 + 2*q2*q3),      dt*(q1**2 - q2**2 + q3**2 - q4**2),                 dt*(-2*q1*q2 + 2*q3*q4),                         0,                 

In [19]:
q_noise = [0.01, 0.01, 0.01, 0.1, 0.1, 0.1]
i = np.eye(6)
Q = Matrix([[val * num for num in i[ind]] for ind, val in enumerate(q_noise)])
G * Q * G.T

Matrix([
[                                                                0.0025*dt**4*(2*q1*q3 + 2*q2*q4)**2 + 0.0025*dt**4*(-2*q1*q4 + 2*q2*q3)**2 + 0.0025*dt**4*(q1**2 + q2**2 - q3**2 - q4**2)**2, 0.0025*dt**4*(-2*q1*q2 + 2*q3*q4)*(2*q1*q3 + 2*q2*q4) + 0.0025*dt**4*(-2*q1*q4 + 2*q2*q3)*(q1**2 - q2**2 + q3**2 - q4**2) + 0.0025*dt**4*(2*q1*q4 + 2*q2*q3)*(q1**2 + q2**2 - q3**2 - q4**2), 0.0025*dt**4*(2*q1*q2 + 2*q3*q4)*(-2*q1*q4 + 2*q2*q3) + 0.0025*dt**4*(-2*q1*q3 + 2*q2*q4)*(q1**2 + q2**2 - q3**2 - q4**2) + 0.0025*dt**4*(2*q1*q3 + 2*q2*q4)*(q1**2 - q2**2 - q3**2 + q4**2),                                                                 0.005*dt**3*(2*q1*q3 + 2*q2*q4)**2 + 0.005*dt**3*(-2*q1*q4 + 2*q2*q3)**2 + 0.005*dt**3*(q1**2 + q2**2 - q3**2 - q4**2)**2, 0.005*dt**3*(-2*q1*q2 + 2*q3*q4)*(2*q1*q3 + 2*q2*q4) + 0.005*dt**3*(-2*q1*q4 + 2*q2*q3)*(q1**2 - q2**2 + q3**2 - q4**2) + 0.005*dt**3*(2*q1*q4 + 2*q2*q3)*(q1**2 + q2**2 - q3**2 - q4**2), 0.005*dt**3*(2*q1*q2 + 2*q3*q4)*(-2*q1*q4 + 2*

In [20]:
P_hat_ = sympy.eye(10) * 0.1
P_hat = F * P_hat_ * F.T + G * Q * G.T
P_hat

Matrix([
[                                                                                                                                                                                                0.0025*dt**4*(2*q1*q3 + 2*q2*q4)**2 + 0.0025*dt**4*(-2*q1*q4 + 2*q2*q3)**2 + 0.025*dt**4*(2*a_{x}*q1 - 2*a_{y}*q4 + 2*a_{z}*q3)**2 + 0.025*dt**4*(2*a_{x}*q2 + 2*a_{y}*q3 + 2*a_{z}*q4)**2 + 0.025*dt**4*(-2*a_{x}*q3 + 2*a_{y}*q2 + 2*a_{z}*q1)**2 + 0.025*dt**4*(-2*a_{x}*q4 - 2*a_{y}*q1 + 2*a_{z}*q2)**2 + 0.0025*dt**4*(q1**2 + q2**2 - q3**2 - q4**2)**2 + 0.1*dt**2 + 0.1, 0.0025*dt**4*(-2*q1*q2 + 2*q3*q4)*(2*q1*q3 + 2*q2*q4) + 0.0025*dt**4*(-2*q1*q4 + 2*q2*q3)*(q1**2 - q2**2 + q3**2 - q4**2) + 0.0025*dt**4*(2*q1*q4 + 2*q2*q3)*(q1**2 + q2**2 - q3**2 - q4**2) + 0.025*dt**4*(2*a_{x}*q1 - 2*a_{y}*q4 + 2*a_{z}*q3)*(-2*a_{x}*q4 - 2*a_{y}*q1 + 2*a_{z}*q2) + 0.025*dt**4*(2*a_{x}*q1 - 2*a_{y}*q4 + 2*a_{z}*q3)*(2*a_{x}*q4 + 2*a_{y}*q1 - 2*a_{z}*q2) + 0.025*dt**4*(2*a_{x}*q2 + 2*a_{y}*q3 + 2*a_{z}*q4)*(

In [21]:
P_hat.evalf(subs=values)

Matrix([
[          0.10101025,                   0,                    0,              0.010205,                    0,                     0,  0.00099906264647522,  -2.49921882323892e-5,  2.49921882323892e-5,  -2.49921882323892e-5],
[                   0,          0.10101025,                    0,                     0,             0.010205,                     0,  2.49921882323892e-5,   2.49921882323892e-5,  2.49921882323892e-5,   0.00099906264647522],
[                   0,                   0,           0.10101025,                     0,                    0,              0.010205,  2.49921882323892e-5,  -2.49921882323892e-5, -0.00099906264647522,   2.49921882323892e-5],
[            0.010205,                   0,                    0,                0.1041,                    0,                     0,   0.0199812529295044, -0.000499843764647784, 0.000499843764647784, -0.000499843764647784],
[                   0,            0.010205,                    0,                     0,   

In [22]:
class ExtendedKalmanFilter_setup1:
    """Extended Kalman Filter
    for vehicle whose motion is modeled as eq. (5.9) in [1]
    and with observation of its 2d location (x, y)
    """
    x = None
    P = None
    R = None
    px = symbols('p_{x}')
    py = symbols('p_{y}')
    pz = symbols('p_{z}')
    
    vx = symbols('v_{x}')
    vy = symbols('v_{y}')
    vz = symbols('v_{z}')
    
    q1 = symbols('q1')
    q2 = symbols('q2')
    q3 = symbols('q3')
    q4 = symbols('q4')
    
    ax = symbols('a_{x}')
    ay = symbols('a_{y}')
    az = symbols('a_{z}')
    wx = symbols('w_{x}')
    wy = symbols('w_{y}')
    wz = symbols('w_{z}')

    dt = symbols('dt')
    norm_w = symbols('\|w\|')
    aR1 = symbols('aR_{1}')
    aR2 = symbols('aR_{2}')
    aR3 = symbols('aR_{3}')
    qk = Matrix([
        [1],[0],[0],[0]
    ])
    g = Matrix([
        [0],[0],[9.81]
    ])

    errors = []
    
    def __init__(self, x, P, H):
        """ 
        Args:
            x (numpy.array): state to estimate: [x_, y_, theta]^T
            P (numpy.array): estimation error covariance
        """
        
        R = Matrix([[self.q1**2 + self.q2**2 - self.q3**2 + self.q4**2, 
                     2*(self.q2*self.q3 - self.q1*self.q4), 
                     2*(self.q1*self.q3 + self.q2*self.q4)],
                    [2*(self.q2*self.q3 + self.q1*self.q4), 
                     self.q1**2 - self.q2**2 + self.q3**2 - self.q4**2, 
                     2*(self.q3*self.q4 - self.q1*self.q2)],
                    [2*(self.q2*self.q4 - self.q1*self.q3), 2*(self.q1*self.q2 + self.q3*self.q4), 
                     self.q1**2 - self.q2**2 - self.q3**2 + self.q4**2]
           ])
        Omega = Matrix([
            [0, self.wz, -self.wy, self.wx],
            [-self.wz, 0, self.wx, self.wy],
            [self.wy, -self.wx, 0, self.wz],
            [-self.wx, -self.wy, -self.wz, 0]
        ])
        # Omega = Matrix([
        #     [0, -self.wx, -self.wy, -self.wz],
        #     [self.wz, 0, self.wx, -self.wy],
        #     [self.wy, -self.wz, 0, self.wx],
        #     [self.wz, self.wy, -self.wx, 0]
        # ])
        A = sympy.cos(self.norm_w*self.dt/2) * sympy.eye(4)
        B = (1/self.norm_w)*sympy.sin(self.norm_w*self.dt/2) * Omega
        # A = sympy.eye(4)
        # B = self.dt / 2 * Omega
        # self.fxu = Matrix([
        #     [self.px + self.vx*self.dt + 1/2 * (R[0,0]*self.ax-self.g[0] + R[0,1]*self.ay-self.g[1] + R[0,2]*self.az-self.g[2])*self.dt**2],
        #     [self.py + self.vy*self.dt + 1/2 * (R[1,0]*self.ax-self.g[0] + R[1,1]*self.ay-self.g[1] + R[1,2]*self.az-self.g[2])*self.dt**2],
        #     [self.pz + self.vz*self.dt + 1/2 * (R[2,0]*self.ax-self.g[0] + R[2,1]*self.ay-self.g[1] + R[2,2]*self.az-self.g[2])*self.dt**2],
        #     [self.vx + (R[0,0]*self.ax-self.g[0] + R[0,1]*self.ay-self.g[1] + R[0,2]*self.az-self.g[2])*self.dt],
        #     [self.vy + (R[1,0]*self.ax-self.g[0] + R[1,1]*self.ay-self.g[1] + R[1,2]*self.az-self.g[2])*self.dt],
        #     [self.vz + (R[2,0]*self.ax-self.g[0] + R[2,1]*self.ay-self.g[1] + R[2,2]*self.az-self.g[2])*self.dt],
        #     [(A + B) * self.qk],
        # ])
        self.fxu = Matrix([
            [Matrix([[self.px],[self.py],[self.pz]]) +  Matrix([[self.vx],[self.vy],[self.vz]]) * self.dt + (R * Matrix([[self.ax],[self.ay], [self.az]]) - self.g)*self.dt**2 / 2],
            [Matrix([[self.vx],[self.vy],[self.vz]]) + (R * Matrix([[self.ax],[self.ay],[self.az]]) - self.g) * self.dt],
            [(A + B) * Matrix([[q1],[q2],[q3],[q4]])],
        ])
        state_x = Matrix([self.px, self.py, self.pz, self.vx, self.vy, self.vz, self.q1, self.q2, self.q3, self.q4])
        control_input = Matrix([self.ax, self.ay, self.az, self.wx, self.wy, self.wz])

        self.F = self.fxu.jacobian(state_x)
        self.G = self.fxu.jacobian(control_input)
        self.P = P
        self.H = H
        self.x = x

    def compute_norm_w(self, wx_, wy_, wz_):
        return np.sqrt(wx_**2 + wy_**2 + wz_**2)
        
    def predict(self, u, dt, Q):
        """estimate x and P based on previous stete of x and control input u
        Args:
            u  (numpy.array): control input u
            dt (numpy.array): difference of current time and previous time
            Q  (numpy.array): process noise 
        """
        # propagate state x
        px_, py_, pz_ = self.x[:3, 0]
        vx_, vy_, vz_ = self.x[3:6, 0]
        q1_, q2_, q3_, q4_ = self.x[6:, 0]
        ax_, ay_, az_, wx_, wy_, wz_ = u
        norm_w_ = self.compute_norm_w(wx_, wy_, wz_);
        fxu_values = {
            self.dt: dt,
            self.px: px_,
            self.py: py_,
            self.pz: pz_,
            self.vx: vx_,
            self.vy: vy_,
            self.vz: vz_,
            self.q1: q1_, 
            self.q2: q2_, 
            self.q3: q3_, 
            self.q4: q4_,
            self.ax: ax_,
            self.ay: ay_,
            self.az: az_,
            self.wx: wx_,
            self.wy: wy_, 
            self.wz: wz_,
            self.norm_w: norm_w_
        }
        # predict state vector x
        self.x = np.array(self.fxu.evalf(subs=fxu_values)).astype(float)

        # predict state covariance matrix P
        P_hat = self.F * self.P * self.F.T + self.G * Matrix(Q) * self.G.T
        self.P = np.array(P_hat.evalf(subs=fxu_values)).astype(float)
        
    def update(self, z, R):
        """update x and P based on observation of (x_, y_)
        Args:
            z (numpy.array): measurement for [x_, y_]^T
            R (numpy.array): measurement noise covariance
        """
        # compute Kalman gain
        K = self.P @ self.H.T @ np.linalg.inv(self.H @ self.P @ self.H.T + R)
        # update state x
        z_ = np.dot(self.H, self.x)  # expected observation from the estimated state
        self.errors.append(np.sqrt(np.sum((z-z_)**2)))
        self.x = self.x + K @ (z - z_)

        # update covariance P
        self.P = self.P - K @ self.H @ self.P

    def plot_error(self):
        plt.plot([i for i in range(len(self.errors))], self.errors, label='Error', color='r')